## Code to calculate the auto and manual difference using an objective function

Weighted least squares solution

### Import packages, functions, manual and automated data

In [47]:
import pandas as pd
import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
import numpy.ma as ma
import datetime
import math

manual_path = '/media/jukes/jukes1/Manual/'; manual_filename = 'manual_tpos.csv'
auto_path = '/home/jukes/Documents/Sample_glaciers/'

In [48]:
os.chdir('/home/jukes/automated-glacier-terminus') #import necessary functions:
from automated_terminus_functions import distance

In [82]:
# MANUAL info
condition_df = pd.read_csv(manual_path+'LS8_manual_delineation_info.csv', dtype=str)

# TEST images
test_df = pd.read_csv(manual_path+'train.csv', dtype=str, header=None)
test_df = test_df.rename(columns={0: 'BoxID', 1: 'datetimes', 2: 'Scene'})
test_df.head()

,BoxID,datetimes,Scene
0,002,2016-03-19,LC80320052016079LGN00
1,120,2016-09-11,LC82330152016255LGN00
2,002,2014-03-28,LC80340052014087LGN00
3,002,2013-08-27,LC80310052013239LGN00
4,120,2016-12-25,LC82320182016360LGN00


In [83]:
# examine_df = condition_df.merge(test_df, how='inner', on=['datetimes', 'Scene', 'BoxID'])
# examine_df 

In [84]:
#MANUAL TERMINUS POSITIONS
manual_df = pd.read_csv(manual_path+manual_filename, dtype=str,sep=',')

#SPLIT INTO 3 DATAFRAMES FOR 3 FLOWLINES:
manual50 = manual_df[['BoxID','datetimes', 'intersect_x', 'intersect_y', 
                                      'tpos50']].copy().reset_index(drop=True).rename(columns={"tpos50": "tpos"})
manual25 = manual_df[['BoxID','datetimes', 'intersect_x', 'intersect_y', 
                                      'tpos25']].copy().reset_index(drop=True).rename(columns={"tpos25": "tpos"})
manual75 = manual_df[['BoxID','datetimes', 'intersect_x', 'intersect_y',
                                      'tpos75']].copy().reset_index(drop=True).rename(columns={"tpos75": "tpos"})
# manual_df.head()

In [85]:
newIDs = []
for item in np.array(condition_df['BoxID']):
    if type(item) != float:
        newIDs.append(item.rjust(3, '0'))
    else:
        newIDs.append('NaN')
condition_df['BoxID'] = newIDs 
condition_df

,Unnamed: 0,BoxID,datetimes,Path,Row,Scene,Condition,Not_exact_date,Jukes
0,21,001,2013-05-03,35,5,LC80350052013123LGN01,NaN,NaN,Jackie
1,24,001,2013-05-05,33,5,LC80330052013125LGN01,Sea ice,NaN,Delineation rate (2 ppl):
2,27,001,2013-05-14,32,5,LC80320052013134LGN03,Sea ice,NaN,1 - 2.5 hrs / 160 lines = ~1.9 min/line
3,37,001,2013-05-28,34,5,LC80340052013148LGN00,Sea ice,2013-05-29,2 - 3.25 hrs / 168 lines = ~2.3 min/line
4,61,001,2013-08-23,35,5,LC80350052013235LGN00,Clear,NaN,120 -
5,67,001,2013-08-27,31,5,LC80310052013239LGN00,Clear,NaN,^ Jukes: 1 hr / 35 lines = 1.7 min/line
6,72,001,2013-09-03,32,5,LC80320052013246LGN00,Thin clouds,NaN,Jackie: 37 mins / 32 lines = 1.2 min/line
7,86,001,2013-09-10,33,5,LC80330052013253LGN00,Clear,NaN,174 -
8,103,001,2013-09-15,36,4,LC80360042013258LGN00,Too cloudy,NaN,Jackie: 44 mins/ 28 lines = 1.6 min/line
9,105,001,2013-09-17,34,5,LC80340052013260LGN00,Clear,NaN,259 -


In [86]:
# TEST image conditions by condition - manual
merge1 = manual_df.merge(condition_df, how='inner', on=['datetimes', 'BoxID']).drop(['Unnamed: 0_x', 
                                                                 'Unnamed: 0_y',
                                                                 'Line_x', 'Line_y', 
                                                                 'Jukes', 'Not_exact_date'], axis=1)
merge2 = merge1.merge(test_df, how='inner', on=['datetimes', 'BoxID', 'Scene'])
merge2

,BoxID,datetimes,intersect_x,intersect_y,tpos50,tpos25,tpos75,Path,Row,Scene,Condition
0,001,2013-05-05,142,159,390.0,270.0,465.0,33,5,LC80330052013125LGN01,Sea ice
1,001,2013-05-14,142,159,390.0,240.0,450.0,32,5,LC80320052013134LGN03,Sea ice
2,001,2013-08-23,140,159,360.0,210.0,465.0,35,5,LC80350052013235LGN00,Clear
3,001,2013-08-27,140,159,360.0,180.0,435.0,31,5,LC80310052013239LGN00,Clear
4,001,2013-09-10,139,158,345.0,195.0,450.0,33,5,LC80330052013253LGN00,Clear
5,001,2013-09-26,137,158,315.0,165.0,420.0,33,5,LC80330052013269LGN00,Clear
6,001,2014-03-14,139,158,345.0,210.0,435.0,32,5,LC80320052014073LGN00,Sea ice
7,001,2014-03-19,141,159,375.0,225.0,450.0,35,5,LC80350052014078LGN00,Sea ice
8,001,2014-03-28,141,158,375.0,210.0,435.0,34,5,LC80340052014087LGN00,Sea ice
9,001,2014-04-08,141,158,375.0,225.0,435.0,31,5,LC80310052014098LGN00,Sea ice


In [87]:
# # look at 002 specifically
# BoxID = '002'
# auto50 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline50_filtered.csv', dtype=str,sep=',')
# auto50 = auto50[['BoxID','datetimes', 'Scene', 'tpos']].copy()
# auto25 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline25_filtered.csv', dtype=str,sep=',')
# auto25 = auto25[['BoxID','datetimes', 'Scene', 'tpos']].copy()
# auto75 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline75_filtered.csv', dtype=str,sep=',')
# auto75 = auto75[['BoxID','datetimes', 'Scene', 'tpos']].copy()

In [88]:
# auto75[auto75['datetimes']=='2016-07-07']

In [92]:
BoxIDs = ['001', '002', '120', '174', '259']
# BoxIDs = ['002']
dfs = []

for BoxID in BoxIDs:
    auto50 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline50_filtered.csv', dtype=str,sep=',')
    auto50 = auto50[['BoxID','datetimes', 'Scene', 'tpos']].copy()
    auto25 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline25_filtered.csv', dtype=str,sep=',')
    auto25 = auto25[['BoxID','datetimes', 'Scene', 'tpos']].copy()
    auto75 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline75_filtered.csv', dtype=str,sep=',')
    auto75 = auto75[['BoxID','datetimes', 'Scene', 'tpos']].copy()
    autodfs = [auto50, auto25, auto75]

#     manual = merge2[merge2.BoxID == BoxID].copy() # USE MERGE 2 IF SUBCATEGORIES
    manual = merge1[merge1.BoxID == BoxID].copy() # USE MERGE 1 FOR FULL SET
    manual50 = manual[['BoxID','datetimes', 'Scene', 'tpos50', 'Condition']].copy().rename(columns={"tpos50": "tpos"})
    manual25 = manual[['BoxID','datetimes', 'Scene', 'tpos25', 'Condition']].copy().rename(columns={"tpos25": "tpos"})
    manual75 = manual[['BoxID','datetimes', 'Scene', 'tpos75', 'Condition']].copy().rename(columns={"tpos75": "tpos"})
    manualdfs = [manual50, manual25, manual75]

    cdfs = []
    for i in range(0, len(manualdfs)):
        adf = autodfs[i]; mdf = manualdfs[i]
        cdf = mdf.merge(adf, how='inner', on='datetimes')
        cdf = cdf.astype({'tpos_x': 'float', 'tpos_y': 'float'})
        cdf['diff'] = abs(np.array(cdf.tpos_x) - np.array(cdf.tpos_y))
        cdfs.append(cdf)
    dfs.append(pd.concat(cdfs))

In [90]:
# compare_cdf

In [93]:
compare_cdf = pd.concat(dfs)
dates = set(compare_cdf.datetimes)
set(compare_cdf.Condition)

{'Clear',
 'Clear ',
 'Cloudy',
 'Dim',
 'Dim, sea ice ',
 'Sea ice',
 'Sea ice ',
 'Shadow',
 'Shadow, clear',
 'Shadow, sea ice',
 'Shadow, sea ice ',
 'Thin clouds',
 'Thin clouds ',
 'Thin clouds, sea ice'}

In [65]:
# #test
# dimbright_df = compare_cdf[compare_cdf['Condition'] == 'Dim']
# seaice_df = pd.concat([compare_cdf[compare_cdf['Condition'] == 'Sea ice '], compare_cdf[compare_cdf['Condition'] == 'Sea ice']])
# clear_df = compare_cdf[compare_cdf['Condition'] == 'Clear']
# thinclouds_df = compare_cdf[compare_cdf['Condition'] == 'Thin clouds']
# good_df = pd.concat([dimbright_df, clear_df, thinclouds_df])

In [95]:
#all
DB = pd.concat([compare_cdf[compare_cdf['Condition'] == 'Dim'], compare_cdf[compare_cdf['Condition'] == 'Dim, sea ice ']])
CL = pd.concat([compare_cdf[compare_cdf['Condition'] == 'Clear'], compare_cdf[compare_cdf['Condition'] == 'Clear ']])
SI = pd.concat([compare_cdf[compare_cdf['Condition'] == 'Sea ice'], compare_cdf[compare_cdf['Condition'] == 'Sea ice  ']])
TC = pd.concat([compare_cdf[compare_cdf['Condition'] == 'Thin clouds'], compare_cdf[compare_cdf['Condition'] == 'Thin clouds '], compare_cdf[compare_cdf['Condition'] == 'Thin clouds, sea ice']])
SH = pd.concat([compare_cdf[compare_cdf['Condition'] == 'Shadow'], compare_cdf[compare_cdf['Condition'] == 'Shadow, clear'], compare_cdf[compare_cdf['Condition'] == 'Shadow, sea ice'], compare_cdf[compare_cdf['Condition'] == 'Shadow, sea ice ']])

In [110]:
df = TC
# df = df.reset_index(drop=True)
# df

In [111]:
# # drop repeated rows
# df = df.drop([16])
# df

In [112]:
misfit = np.array(df['diff'])
# misfit = [1080.0, 390.0, 45.0, 105.0]
print(len(misfit))
# print(misfit)
# print(np.average(misfit))
print(np.median(misfit))
# print(np.std(misfit))
print("MAD = ", np.median(abs(misfit-np.median(misfit))))
# print(np.std(misfit)/np.average(misfit))

33
15.0
MAD =  15.0


In [78]:
misfit-np.median(misfit)

array([ 0., 15.,  0.])

## Theta calculation

In [82]:
#SIGMAS (DATA ERRORS) ALONG EACH FLOWLINE (FROM INTERANALYST DIFFERENCES)
sigmas = [35.02, 27.65, 30.45]
sigma_avg = np.average(sigmas); print(sigma_avg)

31.040000000000003


In [101]:
theta1s = []; theta2s = []; compare_dfs = []
#FOR EACH GLACIER BOXID:
BoxIDs = list(set(manual_df.BoxID))
for BoxID in BoxIDs:
    print("Box"+BoxID)
    #grab automated tpos
    auto50 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline50_filtered.csv', dtype=str,sep=',')
    auto25 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline25_filtered.csv', dtype=str,sep=',')
    auto75 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline75_filtered.csv', dtype=str,sep=',')
    autodfs = [auto50, auto25, auto75]
    #grab manual tpos that corresponds to just boxID
    manual50_df = manual50[manual50.BoxID == BoxID].copy()
    manual25_df = manual25[manual25.BoxID == BoxID].copy()
    manual75_df = manual75[manual75.BoxID == BoxID].copy()
    manualdfs = [manual50_df, manual25_df, manual75_df]
    #calculate difference in terminus positions along the three flowlines
    lists3 = []; lists3_norm = []
    for i in range(0, len(manualdfs)):
        man = manualdfs[i]; auto = autodfs[i]; sigma = sigmas[i]
        compare_df = man.merge(auto, how='inner', on=['datetimes'])
        #cast terminus positions into float values
        compare_df = compare_df.astype({'tpos_x': 'float', 'tpos_y': 'float'})
        #subtract the absolute value of the difference and put into df as a column named "diff"
        compare_df['diff'] = abs(np.array(compare_df.tpos_x) - np.array(compare_df.tpos_y))  
        compare_df['diff/sigma'] = abs(np.array(compare_df.tpos_x) - np.array(compare_df.tpos_y))/sigma
        lists3.append(list(compare_df['diff']))  
        lists3_norm.append(list(compare_df['diff/sigma']))
    diff_all = lists3[0]+lists3[1]+lists3[2] #list of all the differences between manual and auto
    normalizeddiff_all = lists3_norm[0]+lists3_norm[1]+lists3_norm[2] #list of all the normalized differences
    N = len(diff_all) #number of total intersections
    
    #CALCULATE THETA:
    theta1 = (1.0/N)*np.sum(normalizeddiff_all) #sum of normalized differences along flowlines
    theta2 = (1.0/N)*(np.sum(diff_all)/sigma_avg) #sum of differences normalized by average sigma
    theta1s.append(theta1); theta2s.append(theta2)
    print("Theta values:",theta1, theta2)
    
    compare_dfs.append(compare_df)

Box174
Theta values: 16.2436864750265 14.980670103092782
Box120
Theta values: 5.468603853588577 5.315721649484535
Box001
Theta values: 4.611837997139841 4.4642857142857135
Box259
Theta values: 1.8672395877956394 1.7949189985272458
Box002


ZeroDivisionError: float division by zero

In [102]:
manual_dfs

NameError: name 'manual_dfs' is not defined

In [60]:
list(zip(columns, theta1_for_df, theta2_for_df))

[('Theta_avg', nan, nan),
 ('174', 15.250678739646302, 15.153724923376984),
 ('002', 43.19720044648473, 43.18432633252906),
 ('001', 32.47436749614078, 32.26716320559405),
 ('120', 13.919454254169144, 13.744797471440512),
 ('259', 26.457638700697967, 26.416362722840447)]

In [67]:
#CALCULATE OVERALL THETA and write results to csv
theta1_all = np.average(theta1s)
theta2_all = np.average(theta2s)

#organize data
columns = ['Theta_avg']+BoxIDs
theta1_for_df = [theta1_all]+theta1s
theta2_for_df = [theta2_all]+theta2s
#write to csv
pd.DataFrame(list(zip(columns, theta1_for_df, theta2_for_df)), 
             columns=['ID', 'theta1', 'theta2']).to_csv(manual_path+'thetas.csv', sep=',') 

#ADJUST FILENAME TO INCLUDE PARAMETERS OR SOMETHING

In [80]:
def objective_func(manual_df):
    #SPLIT INTO 3 DATAFRAMES FOR 3 FLOWLINES:
    manual50 = manual_df[['BoxID','datetimes', 'intersect_x', 'intersect_y', 
                                          'tpos50']].copy().reset_index(drop=True).rename(columns={"tpos50": "tpos"})
    manual25 = manual_df[['BoxID','datetimes', 'intersect_x', 'intersect_y', 
                                          'tpos25']].copy().reset_index(drop=True).rename(columns={"tpos25": "tpos"})
    manual75 = manual_df[['BoxID','datetimes', 'intersect_x', 'intersect_y',
                                          'tpos75']].copy().reset_index(drop=True).rename(columns={"tpos75": "tpos"})
    #SIGMAS (DATA ERRORS) ALONG EACH FLOWLINE (FROM INTERANALYST DIFFERENCES)
    sigmas = [35.02, 27.65, 30.45]; sigma_avg = np.average(sigmas);
    
    theta1s = []; theta2s = []
    #FOR EACH GLACIER BOXID:
    BoxIDs = list(set(manual_df.BoxID))
    for BoxID in BoxIDs:
        print("Box"+BoxID)
        #grab automated tpos
        auto50 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline50_filtered.csv', dtype=str,sep=',')
        auto25 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline25_filtered.csv', dtype=str,sep=',')
        auto75 = pd.read_csv(auto_path+'Tpos_Box'+BoxID+'_flowline75_filtered.csv', dtype=str,sep=',')
        autodfs = [auto50, auto25, auto75]
        #grab manual tpos that corresponds to just boxID
        manual50_df = manual50[manual50.BoxID == BoxID].copy()
        manual25_df = manual25[manual25.BoxID == BoxID].copy()
        manual75_df = manual75[manual75.BoxID == BoxID].copy()
        manualdfs = [manual50, manual25, manual75]
        #calculate difference in terminus positions along the three flowlines
        lists3 = []; lists3_norm = []
        for i in range(0, len(manualdfs)):
            man = manualdfs[i]; auto = autodfs[i]; sigma = sigmas[i]
            compare_df = man.merge(auto, how='inner', on=['datetimes'])
            #cast terminus positions into float values
            compare_df = compare_df.astype({'tpos_x': 'float', 'tpos_y': 'float'})
            #subtract the absolute value of the difference and put into df as a column named "diff"
            compare_df['diff'] = abs(np.array(compare_df.tpos_x) - np.array(compare_df.tpos_y))  
            compare_df['diff/sigma'] = abs(np.array(compare_df.tpos_x) - np.array(compare_df.tpos_y))/sigma
            lists3.append(list(compare_df['diff']))  
            lists3_norm.append(list(compare_df['diff/sigma']))
        diff_all = lists3[0]+lists3[1]+lists3[2] #list of all the differences between manual and auto
        normalizeddiff_all = lists3_norm[0]+lists3_norm[1]+lists3_norm[2] #list of all the normalized differences
        N = len(diff_all) #number of total intersections

        #CALCULATE THETA:
        theta1 = (1.0/N)*np.sum(normalizeddiff_all) #sum of normalized differences along flowlines
        theta2 = (1.0/N)*(np.sum(diff_all)/sigma_avg) #sum of differences normalized by average sigma
        theta1s.append(theta1); theta2s.append(theta2)
        #print("Theta values:",theta1, theta2)   
        
    #CALCULATE OVERALL THETA
    theta1_all = np.average(theta1s); theta2_all = np.average(theta2s)
    #organize data in dataframe
    column_titles = ['Theta_avg']+BoxIDs
    theta1_for_df = [theta1_all]+theta1s; theta2_for_df = [theta2_all]+theta2s
    #write to csv
    theta_df = pd.DataFrame(list(zip(column_titles, theta1_for_df, theta2_for_df)), 
                 columns=['ID', 'theta1', 'theta2'])
    return theta_df 

In [78]:
# objective_func(manual_df)